**Inicialmente vamos a importar las librerías necesarias para que el código funcione, en el caso de necesitar librerías nuevas, serán agregadas a medida que sean requeridas**

In [1]:
!pip install kaggle
# 1. Importar las librerías necesarias
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import os
import json
import pandas as pd
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.impute import SimpleImputer


In [2]:
os.environ['KAGGLE_CONFIG_DIR']='/content/'

In [3]:
# Cargar credenciales de Kaggle
data = {"username":"juanmanuellopezarias","key":"3a9f59f3e8a3191d8732a210262ac91a"}
with open('kaggle.json','w') as file:
  json.dump(data, file, indent=4)

**Descargamos los datos de Kaggle**

In [4]:
!kaggle competitions download -c udea-ai4eng-20242

 45% 9.00M/20.1M [00:00<00:00, 64.2MB/s]
100% 20.1M/20.1M [00:00<00:00, 114MB/s] 


In [5]:
!unzip \*.zip && rm *.zip

Archive:  udea-ai4eng-20242.zip
  inflating: submission_example.csv  
  inflating: test.csv                
  inflating: train.csv               


# **Fase de carga de datos e inspección inicial**

In [6]:
#Cargar el archivo CSV
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')
df=train.copy()
df=df.drop(['ID'], axis=1)
df=df.drop(['PERIODO'], axis=1)

#Revisión inicial de los datos
print(df.head())
print(df.info())
print(df.describe())

#Manejo de datos faltantes
#Identificar columnas con valores nulos
print(df.isnull().sum())



          ESTU_PRGM_ACADEMICO ESTU_PRGM_DEPARTAMENTO  \
0                  ENFERMERIA                 BOGOTÁ   
1                     DERECHO              ATLANTICO   
2       MERCADEO Y PUBLICIDAD                 BOGOTÁ   
3  ADMINISTRACION DE EMPRESAS              SANTANDER   
4                  PSICOLOGIA              ANTIOQUIA   

             ESTU_VALORMATRICULAUNIVERSIDAD ESTU_HORASSEMANATRABAJA  \
0  Entre 5.5 millones y menos de 7 millones       Menos de 10 horas   
1  Entre 2.5 millones y menos de 4 millones                       0   
2  Entre 2.5 millones y menos de 4 millones         Más de 30 horas   
3  Entre 4 millones y menos de 5.5 millones                       0   
4  Entre 2.5 millones y menos de 4 millones     Entre 21 y 30 horas   

  FAMI_ESTRATOVIVIENDA FAMI_TIENEINTERNET                 FAMI_EDUCACIONPADRE  \
0            Estrato 3                 Si    Técnica o tecnológica incompleta   
1            Estrato 3                 No      Técnica o tecnológica compl

In [7]:
train_labels = train[['ID', 'RENDIMIENTO_GLOBAL']]

# Eliminar la columna label de train para unirlo con test
train = train.drop(columns=['RENDIMIENTO_GLOBAL'])

# Unir train y test
datos_combinados = pd.concat([train, test], ignore_index=True)

# **Fase de imputación de datos y tratamiento especial de algunas variables categóricas binarias**

In [8]:
#Variables Categoricas
ccols = [i for i in datos_combinados.columns if not i in datos_combinados._get_numeric_data()]
print (ccols)
#Valor mas repetido
valor_comun = datos_combinados[ccols].mode().iloc[0]
valor_comun
#Reemplazar datos faltantes con los mas repetidos
for column in ccols:
  datos_combinados[column].fillna(valor_comun[column], inplace=True)

datos_combinados

['ESTU_PRGM_ACADEMICO', 'ESTU_PRGM_DEPARTAMENTO', 'ESTU_VALORMATRICULAUNIVERSIDAD', 'ESTU_HORASSEMANATRABAJA', 'FAMI_ESTRATOVIVIENDA', 'FAMI_TIENEINTERNET', 'FAMI_EDUCACIONPADRE', 'FAMI_EDUCACIONMADRE', 'ESTU_PAGOMATRICULAPROPIO']


<ipython-input-8-04b137fec62f>:9: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  datos_combinados[column].fillna(valor_comun[column], inplace=True)


,ID,PERIODO,ESTU_PRGM_ACADEMICO,ESTU_PRGM_DEPARTAMENTO,ESTU_VALORMATRICULAUNIVERSIDAD,ESTU_HORASSEMANATRABAJA,FAMI_ESTRATOVIVIENDA,FAMI_TIENEINTERNET,FAMI_EDUCACIONPADRE,FAMI_EDUCACIONMADRE,ESTU_PAGOMATRICULAPROPIO,Unnamed: 0
0,904256,20212,ENFERMERIA,BOGOTÁ,Entre 5.5 millones y menos de 7 millones,Menos de 10 horas,Estrato 3,Si,Técnica o tecnológica incompleta,Postgrado,No,NaN
1,645256,20212,DERECHO,ATLANTICO,Entre 2.5 millones y menos de 4 millones,0,Estrato 3,No,Técnica o tecnológica completa,Técnica o tecnológica incompleta,No,NaN
2,308367,20203,MERCADEO Y PUBLICIDAD,BOGOTÁ,Entre 2.5 millones y menos de 4 millones,Más de 30 horas,Estrato 3,Si,Secundaria (Bachillerato) completa,Secundaria (Bachillerato) completa,No,NaN
3,470353,20195,ADMINISTRACION DE EMPRESAS,SANTANDER,Entre 4 millones y menos de 5.5 millones,0,Estrato 4,Si,No sabe,Secundaria (Bachillerato) completa,No,NaN
4,989032,20212,PSICOLOGIA,ANTIOQUIA,Entre 2.5 millones y menos de 4 millones,Entre 21 y 30 horas,Estrato 3,Si,Primaria completa,Primaria completa,No,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
989281,496981,20195,ADMINISTRACION DE EMPRESAS,BOGOTÁ,Entre 2.5 millones y menos de 4 millones,Más de 30 horas,Estrato 1,Si,Primaria incompleta,Primaria incompleta,Si,296781.0
989282,209415,20183,DERECHO,META,Entre 1 millón y menos de 2.5 millones,0,Estrato 4,Si,Educación profesional completa,Educación profesional completa,No,296782.0
989283,239074,20212,DERECHO,BOGOTÁ,Entre 2.5 millones y menos de 4 millones,Más de 30 horas,Estrato 3,Si,Secundaria (Bachillerato) completa,Educación profesional completa,No,296783.0
989284,963852,20195,INGENIERIA AERONAUTICA,ANTIOQUIA,Entre 5.5 millones y menos de 7 millones,Entre 11 y 20 horas,Estrato 3,Si,Educación profesional completa,Educación profesional completa,No,296784.0


In [9]:
datos_combinados.head()

,ID,PERIODO,ESTU_PRGM_ACADEMICO,ESTU_PRGM_DEPARTAMENTO,ESTU_VALORMATRICULAUNIVERSIDAD,ESTU_HORASSEMANATRABAJA,FAMI_ESTRATOVIVIENDA,FAMI_TIENEINTERNET,FAMI_EDUCACIONPADRE,FAMI_EDUCACIONMADRE,ESTU_PAGOMATRICULAPROPIO,Unnamed: 0
0,904256,20212,ENFERMERIA,BOGOTÁ,Entre 5.5 millones y menos de 7 millones,Menos de 10 horas,Estrato 3,Si,Técnica o tecnológica incompleta,Postgrado,No,NaN
1,645256,20212,DERECHO,ATLANTICO,Entre 2.5 millones y menos de 4 millones,0,Estrato 3,No,Técnica o tecnológica completa,Técnica o tecnológica incompleta,No,NaN
2,308367,20203,MERCADEO Y PUBLICIDAD,BOGOTÁ,Entre 2.5 millones y menos de 4 millones,Más de 30 horas,Estrato 3,Si,Secundaria (Bachillerato) completa,Secundaria (Bachillerato) completa,No,NaN
3,470353,20195,ADMINISTRACION DE EMPRESAS,SANTANDER,Entre 4 millones y menos de 5.5 millones,0,Estrato 4,Si,No sabe,Secundaria (Bachillerato) completa,No,NaN
4,989032,20212,PSICOLOGIA,ANTIOQUIA,Entre 2.5 millones y menos de 4 millones,Entre 21 y 30 horas,Estrato 3,Si,Primaria completa,Primaria completa,No,NaN


In [12]:
#borrar las columnas de FAMI_EDUCACIONPADRE FAMI_EDUCACIONMADRE
# datos_combinados = datos_combinados.drop(['FAMI_EDUCACIONPADRE', 'FAMI_EDUCACIONMADRE'], axis=1)
datos_combinados = datos_combinados.drop(['ESTU_PRGM_ACADEMICO'], axis=1)

 Tratamiento especial de la columna **ESTU_PRGM_ACADEMICO** para reducir las columnas al hacer el One Hot encoding

In [14]:
# def categorizar_programa(academico):
#     academico = academico.upper()  # Convertir a mayúsculas para evitar distinciones de minúsculas/mayúsculas

#     if academico.startswith('INGE'):
#         return 'INGENIERIA'
#     elif academico.startswith('LIC'):
#         return 'LICENCIATURA'
#     elif academico.startswith('ADM'):
#         return 'ADMINISTRACION'
#     elif academico.startswith('CONT'):
#         return 'CONTADURIA'
#     elif academico.startswith('COMU'):
#         return 'COMUNICACION'
#     elif 'DEPORT' in academico:
#         return 'DEPORTE'
#     elif 'PSIC' in academico:
#         return 'PSICOLOGIA'
#     elif 'TRABAJO SOCIAL' in academico:
#         return 'TRABAJO SOCIAL'
#     elif 'DISE' in academico:
#         return 'DISEÑO'
#     elif 'TERAP' in academico:
#         return 'TERAPIA'
#     elif 'GASTRON' in academico:
#         return 'GASTRONOMIA'
#     elif 'ZOOTECNIA' in academico:
#         return 'VETERINARIA Y ZOOTECNIA'
#     elif 'NUTRIC' in academico:
#         return 'NUTRICION'
#     elif 'MARKETING' in academico or 'PUBLICIDAD' in academico or 'MERCAD' in academico:
#         return 'MARKETING'
#     elif 'MATE' in academico or 'ESTA' in academico:
#         return 'MATEMATICAS Y ESTADISTICA'
#     elif 'ARTE' in academico:
#         return 'ARTES'
#     elif 'PERIOD' in academico:
#         return 'PERIODISMO'
#     elif 'LITER' in academico:
#         return 'LITERATURA'
#     elif 'MUS' in academico or 'MÚS' in academico:
#         return 'MUSICA'
#     elif 'ECONO' in academico or 'NEG' in academico or 'COME' in academico or 'FIN' in academico:
#         return 'ECONOMIA Y NEGOCIOS'
#     elif 'GEO' in academico:
#         return 'CIENCIAS DE LA TIERRA'
#     elif 'CINE' in academico or 'RADIO' in academico or 'TELEV' in academico or 'AUDIOV' in academico or 'FOTOG' in academico:
#         return 'CIENCIAS AUDIOVISUALES'
#     elif 'DERECHO' in academico:
#         return 'DERECHO'
#     elif 'POLIT' in academico or 'POLÍT' in academico:
#         return 'CIENCIAS POLITICAS'
#     elif 'QUIM' in academico or 'QUÍM' in academico:
#         return 'QUIMICA'
#     elif 'CONST' in academico:
#         return 'CONSTRUCCION'
#     elif 'TURIS' in academico:
#         return 'TURISMO'
#     elif 'BIBLIO' in academico:
#         return 'BIBLIOTECOLOGIA'
#     elif 'FILOS' in academico:
#         return 'FILOSOFIA'
#     elif 'FILOL' in academico:
#         return 'FILOLOGIA'
#     elif 'ODONTOLOG' in academico:
#         return 'ODONTOLOGIA'
#     elif 'BIOL' in academico:
#         return 'BIOLOGIA Y MICROBIOLOGIA'
#     elif academico.startswith('ECOL'):
#         return 'ECOLOGIA'
#     elif 'BACT' in academico:
#         return 'BACTERIOLOGIA'
#     elif 'CULT' in academico:
#         return 'CIENCIAS CULTURALES'
#     elif 'INSTRUMENTACION' in academico:
#         return 'INSTRUMENTACION QUIRURGICA'
#     elif 'MILIT' in academico or 'NAVALES' in academico or 'NAU' in academico:
#         return 'CIENCIAS MARITIMAS Y AERONAUTICAS'
#     else:
#         return academico  # Si no coincide con ninguna de las condiciones, se deja igual
# #categorizar academico
# datos_combinados['ESTU_PRGM_ACADEMICO'] = datos_combinados['ESTU_PRGM_ACADEMICO'].apply(categorizar_programa)
# # Usamos una lista de patrones que deseamos reemplazar y los eliminamos en una sola llamada
# patterns = ['PROFESIONAL EN ', 'PROFESIONAL DE ', 'PROFESIONAL ', 'PROGRAMA ']
# # Reemplazamos todos los patrones con un espacio vacío usando regex
# datos_combinados['ESTU_PRGM_ACADEMICO'] = datos_combinados['ESTU_PRGM_ACADEMICO'].replace(patterns, '', regex=True)
# # Aplicar la función a la columna
# datos_combinados['ESTU_PRGM_ACADEMICO'] = datos_combinados['ESTU_PRGM_ACADEMICO'].apply(categorizar_programa)

 Tratamiento de las columnas con valores **binarios** *(si no)*, se decide en la columna de **ESTU_HORASSEMANATRABAJA**, convertir a binaria (Trabajo o no )


In [15]:
datos_combinados["FAMI_TIENEINTERNET"] = datos_combinados["FAMI_TIENEINTERNET"].replace({'Si': 1, 'No': 0})
datos_combinados["ESTU_PAGOMATRICULAPROPIO"] = datos_combinados["ESTU_PAGOMATRICULAPROPIO"].replace({'Si': 1, 'No': 0})
datos_combinados['ESTU_HORASSEMANATRABAJA'] = datos_combinados['ESTU_HORASSEMANATRABAJA'].apply(lambda x: 1 if x != 0 else 0)
datos_combinados.head()

<ipython-input-15-a975cb33ae40>:1: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  datos_combinados["FAMI_TIENEINTERNET"] = datos_combinados["FAMI_TIENEINTERNET"].replace({'Si': 1, 'No': 0})
<ipython-input-15-a975cb33ae40>:2: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  datos_combinados["ESTU_PAGOMATRICULAPROPIO"] = datos_combinados["ESTU_PAGOMATRICULAPROPIO"].replace({'Si': 1, 'No': 0})


,ID,PERIODO,ESTU_PRGM_DEPARTAMENTO,ESTU_VALORMATRICULAUNIVERSIDAD,ESTU_HORASSEMANATRABAJA,FAMI_ESTRATOVIVIENDA,FAMI_TIENEINTERNET,ESTU_PAGOMATRICULAPROPIO,Unnamed: 0
0,904256,20212,BOGOTÁ,Entre 5.5 millones y menos de 7 millones,1,Estrato 3,1,0,NaN
1,645256,20212,ATLANTICO,Entre 2.5 millones y menos de 4 millones,1,Estrato 3,0,0,NaN
2,308367,20203,BOGOTÁ,Entre 2.5 millones y menos de 4 millones,1,Estrato 3,1,0,NaN
3,470353,20195,SANTANDER,Entre 4 millones y menos de 5.5 millones,1,Estrato 4,1,0,NaN
4,989032,20212,ANTIOQUIA,Entre 2.5 millones y menos de 4 millones,1,Estrato 3,1,0,NaN


Identificamos las columnas categóricas eliminamos las que tienen un **tratamiento especial** ya realizado

In [16]:
# Identificar columnas categóricas
categorical_columns = datos_combinados.select_dtypes(include=['object']).columns
print(categorical_columns)

Index(['ESTU_PRGM_DEPARTAMENTO', 'ESTU_VALORMATRICULAUNIVERSIDAD',
       'FAMI_ESTRATOVIVIENDA'],
      dtype='object')


# **One-Hot Encoding y Normalización**

Aplicamos **One-Hot Encoding** a las columnas categóricas restantes, normalizamos características numéricas (si aun existen)

In [17]:
# Aplicar OneHotEncoder a las columnas categóricas
encoder = OneHotEncoder(sparse_output=False, drop='first')  # Eliminar una categoría para evitar multicolinealidad
df_encoded = pd.DataFrame(encoder.fit_transform(df[categorical_columns]),
                          columns=encoder.get_feature_names_out(categorical_columns))

#Agregamos las columnas con tratamiento especial anteriormente realizado
df_encoded = pd.concat([datos_combinados['FAMI_TIENEINTERNET'], datos_combinados['ESTU_PAGOMATRICULAPROPIO'], df_encoded], axis=1)

Unificamos los Dataframes creados y codificados, añadiendo la columna **ID y PERIODO** que borramos al inicio para evitar una normalización de estas características numéricas.

# **Unificación y visualización Final de los Datos**

In [18]:
#Concatenar las columnas numéricas escaladas y las categóricas codificadas
df_preprocessed = df_encoded
df_preprocessed = pd.concat([train['ID'], df_preprocessed], axis=1)
df_preprocessed = pd.concat([train['PERIODO'], df_preprocessed], axis=1)
# Cambiar la configuración de pandas para mostrar todas las columnas
pd.set_option('display.max_columns', None)


Mostramos el **Dataframe Final**

In [19]:
df_preprocessed.head()

,PERIODO,ID,FAMI_TIENEINTERNET,ESTU_PAGOMATRICULAPROPIO,ESTU_PRGM_DEPARTAMENTO_ANTIOQUIA,ESTU_PRGM_DEPARTAMENTO_ARAUCA,ESTU_PRGM_DEPARTAMENTO_ATLANTICO,ESTU_PRGM_DEPARTAMENTO_BOGOTÁ,ESTU_PRGM_DEPARTAMENTO_BOLIVAR,ESTU_PRGM_DEPARTAMENTO_BOYACA,ESTU_PRGM_DEPARTAMENTO_CALDAS,ESTU_PRGM_DEPARTAMENTO_CAQUETA,ESTU_PRGM_DEPARTAMENTO_CASANARE,ESTU_PRGM_DEPARTAMENTO_CAUCA,ESTU_PRGM_DEPARTAMENTO_CESAR,ESTU_PRGM_DEPARTAMENTO_CHOCO,ESTU_PRGM_DEPARTAMENTO_CORDOBA,ESTU_PRGM_DEPARTAMENTO_CUNDINAMARCA,ESTU_PRGM_DEPARTAMENTO_GUAVIARE,ESTU_PRGM_DEPARTAMENTO_HUILA,ESTU_PRGM_DEPARTAMENTO_LA GUAJIRA,ESTU_PRGM_DEPARTAMENTO_MAGDALENA,ESTU_PRGM_DEPARTAMENTO_META,ESTU_PRGM_DEPARTAMENTO_NARIÑO,ESTU_PRGM_DEPARTAMENTO_NORTE SANTANDER,ESTU_PRGM_DEPARTAMENTO_PUTUMAYO,ESTU_PRGM_DEPARTAMENTO_QUINDIO,ESTU_PRGM_DEPARTAMENTO_RISARALDA,ESTU_PRGM_DEPARTAMENTO_SAN ANDRES,ESTU_PRGM_DEPARTAMENTO_SANTANDER,ESTU_PRGM_DEPARTAMENTO_SUCRE,ESTU_PRGM_DEPARTAMENTO_TOLIMA,ESTU_PRGM_DEPARTAMENTO_VALLE,ESTU_PRGM_DEPARTAMENTO_VAUPES,ESTU_VALORMATRICULAUNIVERSIDAD_Entre 2.5 millones y menos de 4 millones,ESTU_VALORMATRICULAUNIVERSIDAD_Entre 4 millones y menos de 5.5 millones,ESTU_VALORMATRICULAUNIVERSIDAD_Entre 5.5 millones y menos de 7 millones,ESTU_VALORMATRICULAUNIVERSIDAD_Entre 500 mil y menos de 1 millón,ESTU_VALORMATRICULAUNIVERSIDAD_Menos de 500 mil,ESTU_VALORMATRICULAUNIVERSIDAD_Más de 7 millones,ESTU_VALORMATRICULAUNIVERSIDAD_No pagó matrícula,ESTU_VALORMATRICULAUNIVERSIDAD_nan,FAMI_ESTRATOVIVIENDA_Estrato 2,FAMI_ESTRATOVIVIENDA_Estrato 3,FAMI_ESTRATOVIVIENDA_Estrato 4,FAMI_ESTRATOVIVIENDA_Estrato 5,FAMI_ESTRATOVIVIENDA_Estrato 6,FAMI_ESTRATOVIVIENDA_Sin Estrato,FAMI_ESTRATOVIVIENDA_nan
0,20212.0,904256.0,1,0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
1,20212.0,645256.0,0,0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
2,20203.0,308367.0,1,0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
3,20195.0,470353.0,1,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
4,20212.0,989032.0,1,0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0


In [20]:
train_listos = df_preprocessed[df_preprocessed['ID'].isin(train_labels['ID'])]
test_listos = df_preprocessed[df_preprocessed['ID'].isin(train_labels['ID'])]


train_listos = train_listos.merge(train_labels, on='ID', how='left')

In [21]:
train_listos.describe()

,PERIODO,ID,FAMI_TIENEINTERNET,ESTU_PAGOMATRICULAPROPIO,ESTU_PRGM_DEPARTAMENTO_ANTIOQUIA,ESTU_PRGM_DEPARTAMENTO_ARAUCA,ESTU_PRGM_DEPARTAMENTO_ATLANTICO,ESTU_PRGM_DEPARTAMENTO_BOGOTÁ,ESTU_PRGM_DEPARTAMENTO_BOLIVAR,ESTU_PRGM_DEPARTAMENTO_BOYACA,ESTU_PRGM_DEPARTAMENTO_CALDAS,ESTU_PRGM_DEPARTAMENTO_CAQUETA,ESTU_PRGM_DEPARTAMENTO_CASANARE,ESTU_PRGM_DEPARTAMENTO_CAUCA,ESTU_PRGM_DEPARTAMENTO_CESAR,ESTU_PRGM_DEPARTAMENTO_CHOCO,ESTU_PRGM_DEPARTAMENTO_CORDOBA,ESTU_PRGM_DEPARTAMENTO_CUNDINAMARCA,ESTU_PRGM_DEPARTAMENTO_GUAVIARE,ESTU_PRGM_DEPARTAMENTO_HUILA,ESTU_PRGM_DEPARTAMENTO_LA GUAJIRA,ESTU_PRGM_DEPARTAMENTO_MAGDALENA,ESTU_PRGM_DEPARTAMENTO_META,ESTU_PRGM_DEPARTAMENTO_NARIÑO,ESTU_PRGM_DEPARTAMENTO_NORTE SANTANDER,ESTU_PRGM_DEPARTAMENTO_PUTUMAYO,ESTU_PRGM_DEPARTAMENTO_QUINDIO,ESTU_PRGM_DEPARTAMENTO_RISARALDA,ESTU_PRGM_DEPARTAMENTO_SAN ANDRES,ESTU_PRGM_DEPARTAMENTO_SANTANDER,ESTU_PRGM_DEPARTAMENTO_SUCRE,ESTU_PRGM_DEPARTAMENTO_TOLIMA,ESTU_PRGM_DEPARTAMENTO_VALLE,ESTU_PRGM_DEPARTAMENTO_VAUPES,ESTU_VALORMATRICULAUNIVERSIDAD_Entre 2.5 millones y menos de 4 millones,ESTU_VALORMATRICULAUNIVERSIDAD_Entre 4 millones y menos de 5.5 millones,ESTU_VALORMATRICULAUNIVERSIDAD_Entre 5.5 millones y menos de 7 millones,ESTU_VALORMATRICULAUNIVERSIDAD_Entre 500 mil y menos de 1 millón,ESTU_VALORMATRICULAUNIVERSIDAD_Menos de 500 mil,ESTU_VALORMATRICULAUNIVERSIDAD_Más de 7 millones,ESTU_VALORMATRICULAUNIVERSIDAD_No pagó matrícula,ESTU_VALORMATRICULAUNIVERSIDAD_nan,FAMI_ESTRATOVIVIENDA_Estrato 2,FAMI_ESTRATOVIVIENDA_Estrato 3,FAMI_ESTRATOVIVIENDA_Estrato 4,FAMI_ESTRATOVIVIENDA_Estrato 5,FAMI_ESTRATOVIVIENDA_Estrato 6,FAMI_ESTRATOVIVIENDA_Sin Estrato,FAMI_ESTRATOVIVIENDA_nan
count,692500.000000,692500.000000,692500.000000,692500.000000,692500.000000,692500.000000,692500.000000,692500.00000,692500.000000,692500.000000,692500.000000,692500.000000,692500.000000,692500.000000,692500.000000,692500.000000,692500.000000,692500.000000,692500.000000,692500.000000,692500.000000,692500.000000,692500.000000,692500.000000,692500.000000,692500.000000,692500.000000,692500.000000,692500.000000,692500.000000,692500.000000,692500.000000,692500.000000,692500.000000,692500.000000,692500.000000,692500.000000,692500.000000,692500.000000,692500.000000,692500.000000,692500.000000,692500.000000,692500.000000,692500.000000,692500.000000,692500.000000,692500.000000,692500.000000
mean,20198.366679,494606.130576,0.894069,0.438702,0.120732,0.000851,0.059235,0.40745,0.029789,0.020286,0.016809,0.003840,0.002674,0.016565,0.011955,0.006194,0.017600,0.020243,0.000053,0.014433,0.006900,0.013736,0.011422,0.019428,0.032618,0.001148,0.011883,0.018309,0.000014,0.041629,0.012526,0.017214,0.064387,0.000020,0.184014,0.100702,0.055581,0.113652,0.115903,0.098215,0.028199,0.009079,0.335987,0.304238,0.094605,0.034091,0.018202,0.004749,0.046407
std,10.535037,285585.209455,0.307749,0.496229,0.325816,0.029152,0.236064,0.49136,0.170005,0.140977,0.128554,0.061846,0.051645,0.127633,0.108685,0.078455,0.131492,0.140829,0.007309,0.119268,0.082777,0.116392,0.106263,0.138024,0.177635,0.033863,0.108360,0.134067,0.003800,0.199740,0.111215,0.130070,0.245441,0.004496,0.387496,0.300934,0.229111,0.317388,0.320109,0.297606,0.165542,0.094849,0.472335,0.460084,0.292669,0.181463,0.133682,0.068753,0.210366
min,20183.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,20195.000000,247324.750000,1.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00

In [22]:
test_listos.describe()

,PERIODO,ID,FAMI_TIENEINTERNET,ESTU_PAGOMATRICULAPROPIO,ESTU_PRGM_DEPARTAMENTO_ANTIOQUIA,ESTU_PRGM_DEPARTAMENTO_ARAUCA,ESTU_PRGM_DEPARTAMENTO_ATLANTICO,ESTU_PRGM_DEPARTAMENTO_BOGOTÁ,ESTU_PRGM_DEPARTAMENTO_BOLIVAR,ESTU_PRGM_DEPARTAMENTO_BOYACA,ESTU_PRGM_DEPARTAMENTO_CALDAS,ESTU_PRGM_DEPARTAMENTO_CAQUETA,ESTU_PRGM_DEPARTAMENTO_CASANARE,ESTU_PRGM_DEPARTAMENTO_CAUCA,ESTU_PRGM_DEPARTAMENTO_CESAR,ESTU_PRGM_DEPARTAMENTO_CHOCO,ESTU_PRGM_DEPARTAMENTO_CORDOBA,ESTU_PRGM_DEPARTAMENTO_CUNDINAMARCA,ESTU_PRGM_DEPARTAMENTO_GUAVIARE,ESTU_PRGM_DEPARTAMENTO_HUILA,ESTU_PRGM_DEPARTAMENTO_LA GUAJIRA,ESTU_PRGM_DEPARTAMENTO_MAGDALENA,ESTU_PRGM_DEPARTAMENTO_META,ESTU_PRGM_DEPARTAMENTO_NARIÑO,ESTU_PRGM_DEPARTAMENTO_NORTE SANTANDER,ESTU_PRGM_DEPARTAMENTO_PUTUMAYO,ESTU_PRGM_DEPARTAMENTO_QUINDIO,ESTU_PRGM_DEPARTAMENTO_RISARALDA,ESTU_PRGM_DEPARTAMENTO_SAN ANDRES,ESTU_PRGM_DEPARTAMENTO_SANTANDER,ESTU_PRGM_DEPARTAMENTO_SUCRE,ESTU_PRGM_DEPARTAMENTO_TOLIMA,ESTU_PRGM_DEPARTAMENTO_VALLE,ESTU_PRGM_DEPARTAMENTO_VAUPES,ESTU_VALORMATRICULAUNIVERSIDAD_Entre 2.5 millones y menos de 4 millones,ESTU_VALORMATRICULAUNIVERSIDAD_Entre 4 millones y menos de 5.5 millones,ESTU_VALORMATRICULAUNIVERSIDAD_Entre 5.5 millones y menos de 7 millones,ESTU_VALORMATRICULAUNIVERSIDAD_Entre 500 mil y menos de 1 millón,ESTU_VALORMATRICULAUNIVERSIDAD_Menos de 500 mil,ESTU_VALORMATRICULAUNIVERSIDAD_Más de 7 millones,ESTU_VALORMATRICULAUNIVERSIDAD_No pagó matrícula,ESTU_VALORMATRICULAUNIVERSIDAD_nan,FAMI_ESTRATOVIVIENDA_Estrato 2,FAMI_ESTRATOVIVIENDA_Estrato 3,FAMI_ESTRATOVIVIENDA_Estrato 4,FAMI_ESTRATOVIVIENDA_Estrato 5,FAMI_ESTRATOVIVIENDA_Estrato 6,FAMI_ESTRATOVIVIENDA_Sin Estrato,FAMI_ESTRATOVIVIENDA_nan
count,692500.000000,692500.000000,692500.000000,692500.000000,692500.000000,692500.000000,692500.000000,692500.00000,692500.000000,692500.000000,692500.000000,692500.000000,692500.000000,692500.000000,692500.000000,692500.000000,692500.000000,692500.000000,692500.000000,692500.000000,692500.000000,692500.000000,692500.000000,692500.000000,692500.000000,692500.000000,692500.000000,692500.000000,692500.000000,692500.000000,692500.000000,692500.000000,692500.000000,692500.000000,692500.000000,692500.000000,692500.000000,692500.000000,692500.000000,692500.000000,692500.000000,692500.000000,692500.000000,692500.000000,692500.000000,692500.000000,692500.000000,692500.000000,692500.000000
mean,20198.366679,494606.130576,0.894069,0.438702,0.120732,0.000851,0.059235,0.40745,0.029789,0.020286,0.016809,0.003840,0.002674,0.016565,0.011955,0.006194,0.017600,0.020243,0.000053,0.014433,0.006900,0.013736,0.011422,0.019428,0.032618,0.001148,0.011883,0.018309,0.000014,0.041629,0.012526,0.017214,0.064387,0.000020,0.184014,0.100702,0.055581,0.113652,0.115903,0.098215,0.028199,0.009079,0.335987,0.304238,0.094605,0.034091,0.018202,0.004749,0.046407
std,10.535037,285585.209455,0.307749,0.496229,0.325816,0.029152,0.236064,0.49136,0.170005,0.140977,0.128554,0.061846,0.051645,0.127633,0.108685,0.078455,0.131492,0.140829,0.007309,0.119268,0.082777,0.116392,0.106263,0.138024,0.177635,0.033863,0.108360,0.134067,0.003800,0.199740,0.111215,0.130070,0.245441,0.004496,0.387496,0.300934,0.229111,0.317388,0.320109,0.297606,0.165542,0.094849,0.472335,0.460084,0.292669,0.181463,0.133682,0.068753,0.210366
min,20183.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,20195.000000,247324.750000,1.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00

In [23]:
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, classification_report
from sklearn.preprocessing import StandardScaler

In [24]:
label_encoder = LabelEncoder()
train_listos['RENDIMIENTO_GLOBAL'] = label_encoder.fit_transform(train_listos['RENDIMIENTO_GLOBAL'])

In [25]:
X = train_listos.drop(['RENDIMIENTO_GLOBAL', 'ID'], axis=1)
y = train_listos['RENDIMIENTO_GLOBAL']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [26]:
train_listos.head()

,PERIODO,ID,FAMI_TIENEINTERNET,ESTU_PAGOMATRICULAPROPIO,ESTU_PRGM_DEPARTAMENTO_ANTIOQUIA,ESTU_PRGM_DEPARTAMENTO_ARAUCA,ESTU_PRGM_DEPARTAMENTO_ATLANTICO,ESTU_PRGM_DEPARTAMENTO_BOGOTÁ,ESTU_PRGM_DEPARTAMENTO_BOLIVAR,ESTU_PRGM_DEPARTAMENTO_BOYACA,ESTU_PRGM_DEPARTAMENTO_CALDAS,ESTU_PRGM_DEPARTAMENTO_CAQUETA,ESTU_PRGM_DEPARTAMENTO_CASANARE,ESTU_PRGM_DEPARTAMENTO_CAUCA,ESTU_PRGM_DEPARTAMENTO_CESAR,ESTU_PRGM_DEPARTAMENTO_CHOCO,ESTU_PRGM_DEPARTAMENTO_CORDOBA,ESTU_PRGM_DEPARTAMENTO_CUNDINAMARCA,ESTU_PRGM_DEPARTAMENTO_GUAVIARE,ESTU_PRGM_DEPARTAMENTO_HUILA,ESTU_PRGM_DEPARTAMENTO_LA GUAJIRA,ESTU_PRGM_DEPARTAMENTO_MAGDALENA,ESTU_PRGM_DEPARTAMENTO_META,ESTU_PRGM_DEPARTAMENTO_NARIÑO,ESTU_PRGM_DEPARTAMENTO_NORTE SANTANDER,ESTU_PRGM_DEPARTAMENTO_PUTUMAYO,ESTU_PRGM_DEPARTAMENTO_QUINDIO,ESTU_PRGM_DEPARTAMENTO_RISARALDA,ESTU_PRGM_DEPARTAMENTO_SAN ANDRES,ESTU_PRGM_DEPARTAMENTO_SANTANDER,ESTU_PRGM_DEPARTAMENTO_SUCRE,ESTU_PRGM_DEPARTAMENTO_TOLIMA,ESTU_PRGM_DEPARTAMENTO_VALLE,ESTU_PRGM_DEPARTAMENTO_VAUPES,ESTU_VALORMATRICULAUNIVERSIDAD_Entre 2.5 millones y menos de 4 millones,ESTU_VALORMATRICULAUNIVERSIDAD_Entre 4 millones y menos de 5.5 millones,ESTU_VALORMATRICULAUNIVERSIDAD_Entre 5.5 millones y menos de 7 millones,ESTU_VALORMATRICULAUNIVERSIDAD_Entre 500 mil y menos de 1 millón,ESTU_VALORMATRICULAUNIVERSIDAD_Menos de 500 mil,ESTU_VALORMATRICULAUNIVERSIDAD_Más de 7 millones,ESTU_VALORMATRICULAUNIVERSIDAD_No pagó matrícula,ESTU_VALORMATRICULAUNIVERSIDAD_nan,FAMI_ESTRATOVIVIENDA_Estrato 2,FAMI_ESTRATOVIVIENDA_Estrato 3,FAMI_ESTRATOVIVIENDA_Estrato 4,FAMI_ESTRATOVIVIENDA_Estrato 5,FAMI_ESTRATOVIVIENDA_Estrato 6,FAMI_ESTRATOVIVIENDA_Sin Estrato,FAMI_ESTRATOVIVIENDA_nan,RENDIMIENTO_GLOBAL
0,20212.0,904256.0,1,0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,2
1,20212.0,645256.0,0,0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1
2,20203.0,308367.0,1,0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1
3,20195.0,470353.0,1,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0
4,20212.0,989032.0,1,0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,3


In [27]:
from tqdm import tqdm
from time import sleep  # Solo para simular tiempos de ejecución
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, classification_report
from sklearn.model_selection import train_test_split
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
import time
from sklearn.linear_model import LogisticRegression

start_time = time.time()

logistic_model = LogisticRegression(
    multi_class='multinomial',
    solver='saga',
    max_iter=200,
    n_jobs=-1,
    random_state=42
)

logistic_model.fit(X_train, y_train)
end_time = time.time()



/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:1247: FutureWarning: 'multi_class' was deprecated in version 1.5 and will be removed in 1.7. From then on, it will always use 'multinomial'. Leave it to its default value to avoid this warning.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_sag.py:349: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(


In [28]:
y_pred = logistic_model.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)

print(f'Accuracy: {accuracy}')
print(f'Tiempo de entrenamiento: {end_time - start_time} segundos')


Accuracy: 0.3243537906137184
Tiempo de entrenamiento: 167.62163090705872 segundos


In [29]:
print("Random Forest Accuracy:", accuracy_score(y_test, y_pred))
print("Random Forest Reporte de Clasificacion:\n", classification_report(y_test, y_pred))

Random Forest Accuracy: 0.3243537906137184
Random Forest Reporte de Clasificacion:
               precision    recall  f1-score   support

           0       0.31      0.83      0.45     35165
           1       0.35      0.45      0.39     34573
           2       0.00      0.00      0.00     34259
           3       0.00      0.00      0.00     34503

    accuracy                           0.32    138500
   macro avg       0.17      0.32      0.21    138500
weighted avg       0.17      0.32      0.21    138500



/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


In [30]:
ids = test_listos['ID']

new_data_processed = test_listos.drop('ID', axis=1)

In [31]:
new_data_processed.head()

,PERIODO,FAMI_TIENEINTERNET,ESTU_PAGOMATRICULAPROPIO,ESTU_PRGM_DEPARTAMENTO_ANTIOQUIA,ESTU_PRGM_DEPARTAMENTO_ARAUCA,ESTU_PRGM_DEPARTAMENTO_ATLANTICO,ESTU_PRGM_DEPARTAMENTO_BOGOTÁ,ESTU_PRGM_DEPARTAMENTO_BOLIVAR,ESTU_PRGM_DEPARTAMENTO_BOYACA,ESTU_PRGM_DEPARTAMENTO_CALDAS,ESTU_PRGM_DEPARTAMENTO_CAQUETA,ESTU_PRGM_DEPARTAMENTO_CASANARE,ESTU_PRGM_DEPARTAMENTO_CAUCA,ESTU_PRGM_DEPARTAMENTO_CESAR,ESTU_PRGM_DEPARTAMENTO_CHOCO,ESTU_PRGM_DEPARTAMENTO_CORDOBA,ESTU_PRGM_DEPARTAMENTO_CUNDINAMARCA,ESTU_PRGM_DEPARTAMENTO_GUAVIARE,ESTU_PRGM_DEPARTAMENTO_HUILA,ESTU_PRGM_DEPARTAMENTO_LA GUAJIRA,ESTU_PRGM_DEPARTAMENTO_MAGDALENA,ESTU_PRGM_DEPARTAMENTO_META,ESTU_PRGM_DEPARTAMENTO_NARIÑO,ESTU_PRGM_DEPARTAMENTO_NORTE SANTANDER,ESTU_PRGM_DEPARTAMENTO_PUTUMAYO,ESTU_PRGM_DEPARTAMENTO_QUINDIO,ESTU_PRGM_DEPARTAMENTO_RISARALDA,ESTU_PRGM_DEPARTAMENTO_SAN ANDRES,ESTU_PRGM_DEPARTAMENTO_SANTANDER,ESTU_PRGM_DEPARTAMENTO_SUCRE,ESTU_PRGM_DEPARTAMENTO_TOLIMA,ESTU_PRGM_DEPARTAMENTO_VALLE,ESTU_PRGM_DEPARTAMENTO_VAUPES,ESTU_VALORMATRICULAUNIVERSIDAD_Entre 2.5 millones y menos de 4 millones,ESTU_VALORMATRICULAUNIVERSIDAD_Entre 4 millones y menos de 5.5 millones,ESTU_VALORMATRICULAUNIVERSIDAD_Entre 5.5 millones y menos de 7 millones,ESTU_VALORMATRICULAUNIVERSIDAD_Entre 500 mil y menos de 1 millón,ESTU_VALORMATRICULAUNIVERSIDAD_Menos de 500 mil,ESTU_VALORMATRICULAUNIVERSIDAD_Más de 7 millones,ESTU_VALORMATRICULAUNIVERSIDAD_No pagó matrícula,ESTU_VALORMATRICULAUNIVERSIDAD_nan,FAMI_ESTRATOVIVIENDA_Estrato 2,FAMI_ESTRATOVIVIENDA_Estrato 3,FAMI_ESTRATOVIVIENDA_Estrato 4,FAMI_ESTRATOVIVIENDA_Estrato 5,FAMI_ESTRATOVIVIENDA_Estrato 6,FAMI_ESTRATOVIVIENDA_Sin Estrato,FAMI_ESTRATOVIVIENDA_nan
0,20212.0,1,0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
1,20212.0,0,0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
2,20203.0,1,0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
3,20195.0,1,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
4,20212.0,1,0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0


In [32]:

predictions = logistic_model.predict(new_data_processed)

test_listos['RENDIMIENTO_GLOBAL'] = predictions



<ipython-input-32-af01c636716f>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_listos['RENDIMIENTO_GLOBAL'] = predictions


In [33]:
test_listos.head()

,PERIODO,ID,FAMI_TIENEINTERNET,ESTU_PAGOMATRICULAPROPIO,ESTU_PRGM_DEPARTAMENTO_ANTIOQUIA,ESTU_PRGM_DEPARTAMENTO_ARAUCA,ESTU_PRGM_DEPARTAMENTO_ATLANTICO,ESTU_PRGM_DEPARTAMENTO_BOGOTÁ,ESTU_PRGM_DEPARTAMENTO_BOLIVAR,ESTU_PRGM_DEPARTAMENTO_BOYACA,ESTU_PRGM_DEPARTAMENTO_CALDAS,ESTU_PRGM_DEPARTAMENTO_CAQUETA,ESTU_PRGM_DEPARTAMENTO_CASANARE,ESTU_PRGM_DEPARTAMENTO_CAUCA,ESTU_PRGM_DEPARTAMENTO_CESAR,ESTU_PRGM_DEPARTAMENTO_CHOCO,ESTU_PRGM_DEPARTAMENTO_CORDOBA,ESTU_PRGM_DEPARTAMENTO_CUNDINAMARCA,ESTU_PRGM_DEPARTAMENTO_GUAVIARE,ESTU_PRGM_DEPARTAMENTO_HUILA,ESTU_PRGM_DEPARTAMENTO_LA GUAJIRA,ESTU_PRGM_DEPARTAMENTO_MAGDALENA,ESTU_PRGM_DEPARTAMENTO_META,ESTU_PRGM_DEPARTAMENTO_NARIÑO,ESTU_PRGM_DEPARTAMENTO_NORTE SANTANDER,ESTU_PRGM_DEPARTAMENTO_PUTUMAYO,ESTU_PRGM_DEPARTAMENTO_QUINDIO,ESTU_PRGM_DEPARTAMENTO_RISARALDA,ESTU_PRGM_DEPARTAMENTO_SAN ANDRES,ESTU_PRGM_DEPARTAMENTO_SANTANDER,ESTU_PRGM_DEPARTAMENTO_SUCRE,ESTU_PRGM_DEPARTAMENTO_TOLIMA,ESTU_PRGM_DEPARTAMENTO_VALLE,ESTU_PRGM_DEPARTAMENTO_VAUPES,ESTU_VALORMATRICULAUNIVERSIDAD_Entre 2.5 millones y menos de 4 millones,ESTU_VALORMATRICULAUNIVERSIDAD_Entre 4 millones y menos de 5.5 millones,ESTU_VALORMATRICULAUNIVERSIDAD_Entre 5.5 millones y menos de 7 millones,ESTU_VALORMATRICULAUNIVERSIDAD_Entre 500 mil y menos de 1 millón,ESTU_VALORMATRICULAUNIVERSIDAD_Menos de 500 mil,ESTU_VALORMATRICULAUNIVERSIDAD_Más de 7 millones,ESTU_VALORMATRICULAUNIVERSIDAD_No pagó matrícula,ESTU_VALORMATRICULAUNIVERSIDAD_nan,FAMI_ESTRATOVIVIENDA_Estrato 2,FAMI_ESTRATOVIVIENDA_Estrato 3,FAMI_ESTRATOVIVIENDA_Estrato 4,FAMI_ESTRATOVIVIENDA_Estrato 5,FAMI_ESTRATOVIVIENDA_Estrato 6,FAMI_ESTRATOVIVIENDA_Sin Estrato,FAMI_ESTRATOVIVIENDA_nan,RENDIMIENTO_GLOBAL
0,20212.0,904256.0,1,0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0
1,20212.0,645256.0,0,0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0
2,20203.0,308367.0,1,0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0
3,20195.0,470353.0,1,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0
4,20212.0,989032.0,1,0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0


In [34]:
test = test_listos[['ID', 'RENDIMIENTO_GLOBAL']]
test.head()

,ID,RENDIMIENTO_GLOBAL
0,904256.0,0
1,645256.0,0
2,308367.0,0
3,470353.0,0
4,989032.0,0


In [35]:
mapping = {2: 'medio-alto', 0: 'alto', 1: 'bajo', 3: 'medio-bajo'}

# Aplicar el mapeo a la columna RENDIMIENTO_GLOBAL
test['RENDIMIENTO_GLOBAL'] = test['RENDIMIENTO_GLOBAL'].map(mapping)

<ipython-input-35-3fb4d68f0492>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test['RENDIMIENTO_GLOBAL'] = test['RENDIMIENTO_GLOBAL'].map(mapping)


In [36]:
test.head()

,ID,RENDIMIENTO_GLOBAL
0,904256.0,alto
1,645256.0,alto
2,308367.0,alto
3,470353.0,alto
4,989032.0,alto


In [37]:
test.to_csv('test_segundomodelo.csv', index=False)